In [2]:
import re
import pandas as pd
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [3]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News333.xlsx")
news.head()

,Title,Body,Category,MegaCategory
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial


In [4]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['am', 'pm'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [7]:

news['clean_title'] = news['Title'].apply(remove_stop_words) 
news['clean_Body'] = news['Body'].apply(remove_stop_words) 
#remove_stop_words_Body


In [8]:
news.head()

,Title,Body,Category,MegaCategory,clean_title,clean_Body
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial,BS-VI transition may lead dumping old stock : ...,BS-VI transition may lead dumping old stock : ...
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial,Annual Report 2016-2017 Bajaj Finserv Limited,broadly described annual report Bajaj Finserv ...
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial,Annual Report 2017-2018 Eicher Motors Limited ...,broadly described annual report Eicher Motors ...
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial,Hindalco Industries : Chairman Kumar Mangalam ...,( enter multiple email addresses separated com...
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial,"WIPRO continues 's uptrend , although weaker note","WIPRO continues 's uptrend , although weaker n..."


In [55]:
# ## Split data
print("\nSplitting data")

title_tr, title_te, MegaCategory_tr, MegaCategory_te = train_test_split(news['clean_title'], news.MegaCategory,test_size =.02)

title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.02)

print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2379
Developement:  49
Testing:  50


In [56]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(MegaCategory_tr)
Ytr = encoder.transform(MegaCategory_tr)
Yde = encoder.transform(MegaCategory_de)
Yte = encoder.transform(MegaCategory_te)


Vectorizing data


In [57]:
print(Ytr)

[11  5  8 ...  4  5  6]


In [58]:
Xtr

<2379x5335 sparse matrix of type '<class 'numpy.int64'>'
	with 20076 stored elements in Compressed Sparse Row format>

In [59]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)
dc_pred = dc.predict(Xde)



Training baseline classifier


In [60]:
print(classification_report(Yde, dc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dc_pred == Yde)))

                      precision    recall  f1-score   support

Awards & Recognition       0.50      0.45      0.48        22
   Corporate Updates       0.00      0.00      0.00         5
               Event       0.00      0.00      0.00         2
              Events       0.00      0.00      0.00         0
           Financial       0.00      0.00      0.00         5
              Growth       0.11      0.14      0.12         7
       Industry News       0.50      0.45      0.48        22
         Market Buzz       0.25      0.17      0.20         6
       Negative News       0.11      0.14      0.12         7
              People       0.50      0.45      0.48        22
   Stock Performance       0.11      0.14      0.12         7
  Uncategorised News       0.50      0.45      0.48        22

           micro avg       0.39      0.37      0.38     28070
           macro avg       0.25      0.23      0.24     28070
        weighted avg       0.41      0.37      0.39     28070

Accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [61]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
dt_pred = dt.predict(Xde)
print(classification_report(Yde, dt_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dt_pred == Yde)))

Training Decision tree
                      precision    recall  f1-score   support

Awards & Recognition       0.76      0.73      0.74        22
   Corporate Updates       0.12      0.20      0.15         5
               Event       0.25      0.50      0.33         2
              Events       0.00      0.00      0.00         0
           Financial       0.12      0.20      0.15         5
              Growth       0.57      0.57      0.57         7
       Industry News       0.76      0.73      0.74        22
         Market Buzz       0.60      0.50      0.55         6
       Negative News       0.57      0.57      0.57         7
              People       0.76      0.73      0.74        22
   Stock Performance       0.57      0.57      0.57         7
  Uncategorised News       0.76      0.73      0.74        22

           micro avg       0.67      0.67      0.67     28070
           macro avg       0.57      0.57      0.56     28070
        weighted avg       0.69      0.67    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [62]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                      precision    recall  f1-score   support

Awards & Recognition       0.69      0.91      0.78        22
   Corporate Updates       0.33      0.20      0.25         5
               Event       0.50      0.50      0.50         2
              Events       0.00      0.00      0.00         0
           Financial       0.33      0.20      0.25         5
              Growth       0.50      0.57      0.53         7
       Industry News       0.69      0.91      0.78        22
         Market Buzz       0.60      0.50      0.55         6
       Negative News       0.50      0.57      0.53         7
              People       0.69      0.91      0.78        22
   Stock Performance       0.50      0.57      0.53         7
  Uncategorised News       0.69      0.91      0.78        22

           micro avg       0.66      0.80      0.72     28070
           macro avg       0.59      0.63      0.59     28070
        weighted avg       0.65      0.80    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [63]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_nb == Yde)))

Training Multinomial Naive Bayesian
                      precision    recall  f1-score   support

Awards & Recognition       0.67      0.82      0.73        22
   Corporate Updates       0.50      0.60      0.55         5
               Event       0.00      0.00      0.00         2
              Events       0.00      0.00      0.00         0
           Financial       0.50      0.60      0.55         5
              Growth       0.60      0.86      0.71         7
       Industry News       0.67      0.82      0.73        22
         Market Buzz       0.50      0.33      0.40         6
       Negative News       0.60      0.86      0.71         7
              People       0.67      0.82      0.73        22
   Stock Performance       0.60      0.86      0.71         7
  Uncategorised News       0.67      0.82      0.73        22

           micro avg       0.64      0.76      0.70     28070
           macro avg       0.53      0.59      0.54     28070
        weighted avg       0.63 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [64]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
svc_pred = svc.predict(Xde)
print(classification_report(Yde, svc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(svc_pred == Yde)))

Training Support Vector Classification


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


                      precision    recall  f1-score   support

Awards & Recognition       0.45      1.00      0.62        22
   Corporate Updates       0.00      0.00      0.00         5
               Event       0.00      0.00      0.00         2
              Events       0.00      0.00      0.00         0
           Financial       0.00      0.00      0.00         5
              Growth       0.00      0.00      0.00         7
       Industry News       0.45      1.00      0.62        22
         Market Buzz       0.00      0.00      0.00         6
       Negative News       0.00      0.00      0.00         7
              People       0.45      1.00      0.62        22
   Stock Performance       0.00      0.00      0.00         7
  Uncategorised News       0.45      1.00      0.62        22

           micro avg       0.45      0.77      0.57     28070
           macro avg       0.19      0.41      0.26     28070
        weighted avg       0.35      0.77      0.48     28070

Accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [65]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
mlp_pred = mlp.predict(Xde)
print(classification_report(Yde, mlp_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == Yde)))

Training Multilayered Perceptron
                      precision    recall  f1-score   support

Awards & Recognition       0.77      0.77      0.77        22
   Corporate Updates       0.40      0.40      0.40         5
               Event       1.00      0.50      0.67         2
              Events       0.00      0.00      0.00         0
           Financial       0.40      0.40      0.40         5
              Growth       0.60      0.86      0.71         7
       Industry News       0.77      0.77      0.77        22
         Market Buzz       0.43      0.50      0.46         6
       Negative News       0.60      0.86      0.71         7
              People       0.77      0.77      0.77        22
   Stock Performance       0.60      0.86      0.71         7
  Uncategorised News       0.77      0.77      0.77        22

           micro avg       0.71      0.74      0.73     28070
           macro avg       0.66      0.65      0.64     28070
        weighted avg       0.72    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [66]:
print('Accuracy achieved is ' + str(np.mean(dc_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(dt_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(pred_nb   == Yde)))
print('Accuracy achieved is ' + str(np.mean(svc_pred  == Yde)))
print('Accuracy achieved is ' + str(np.mean(mlp_pred  == Yde)))

Accuracy achieved is 0.24489795918367346
Accuracy achieved is 0.5714285714285714
Accuracy achieved is 0.6122448979591837
Accuracy achieved is 0.4489795918367347
Accuracy achieved is 0.6530612244897959


In [67]:
# # Final Model: Multilayered Perceptron
# ## Predict test data
print("\n\nPredicting test data using Multilayered Perceptron")
pred_final = mlp.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(accuracy_score(Yte,pred_final)))



Predicting test data using Multilayered Perceptron
                      precision    recall  f1-score   support

Awards & Recognition       0.64      0.82      0.72        22
   Corporate Updates       0.50      0.50      0.50         6
               Event       0.50      0.33      0.40         3
              Events       0.00      0.00      0.00         0
           Financial       0.50      0.50      0.50         6
              Growth       1.00      0.57      0.73         7
       Industry News       0.64      0.82      0.72        22
         Market Buzz       0.33      0.25      0.29         4
       Negative News       1.00      0.57      0.73         7
              People       0.64      0.82      0.72        22
   Stock Performance       1.00      0.57      0.73         7
  Uncategorised News       0.64      0.82      0.72        22

           micro avg       0.64      0.74      0.69     27941
           macro avg       0.58      0.57      0.56     27941
        weighte

In [68]:
output = {"title":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}

In [69]:
df = pd.DataFrame(output, columns=["title","predicted","actual"])
df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_predication_1.csv")